In [16]:
#필요한 라이브러리 정의
#!pip install gradio
import gradio as gr
#!pip install gtts

from tts_final import tts
#!pip install ultralytics
import torch
from ultralytics import YOLO
import cv2
import numpy as np

In [17]:
model = YOLO("path_to_yolov8_model.pt")

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_yolov8_model.pt'

In [19]:
# 영어로 설정된 class를 한국어로 번역 (tts를 위해서)
from collections import Counter

def wordChanger(word):
    if word == "can":
        return "캔"
    elif word == "glass":
        return "유리"
    elif word == "paper":
        return "종이"
    elif word == "pet":
        return "페트병"
    elif word == "plastic":
        return "플라스틱"
    elif word == "styrofoam":
        return "스티로폼"
    elif word == "vinyl":
        return "비닐"
    else:
        return "탐지 불가"

# 최빈값을 찾아 한국어로 번역하는 함수
def bestResult(results):
    # 최빈값 찾기
    counter = Counter(results)
    most_common_word, _ = counter.most_common(1)[0]

    # 최빈값 번역하기
    best = wordChanger(most_common_word)

    return best


In [15]:
# Gradio에서 동작을 수행하기 위한 함수 구현부

# 객체 감지 함수 구현
def objDetect(image):
    """
    주어진 이미지에서 객체를 감지하는 함수.

    매개변수:
    image (ndarray): 감지할 이미지.

    반환:
    results (list): 감지된 객체의 결과 리스트.
    """
    # 예측 수행
    #results = model.predict(image)
    results = ["can", "glass", "can", "paper", "can", "pet", "plastic", "styrofoam", "vinyl", "can"]

    return bestResult(results)

# 이미지에서 객체를 감지하고 TTS로 변환하는 함수
def process_image(image):
    detected_objects = objDetect(image)
    tts_result = tts(detected_objects)
    return tts_result

# Gradio 인터페이스 생성
with gr.Blocks(css="styles.css") as iface:
    # 타이틀 및 설명 텍스트 추가
    gr.Markdown("<div class='container'><h1>Voicena 보이시네</h1>")
    gr.Markdown("사진을 등록해주세요.</div>")

    with gr.Column():
        input_image = gr.Image(type="pil", label="Input Image")
        output_audio = gr.Audio(label="TTS Result", type="filepath", autoplay=True)
        submit_button = gr.Button("Process Image")

    submit_button.click(fn=process_image, inputs=input_image, outputs=output_audio)

    # 하단바 추가
    gr.Markdown("<div class='container'><hr>")
    gr.Markdown("© 2023 Your Company Name. All rights reserved.</div>")

# 인터페이스 실행 및 공유 URL 생성
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a93875bb95ab243791.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
